## Building a Custom Optimizer

In [4]:
%matplotlib inline

# imports 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from openTSNE import TSNE

In [5]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
# preprocessing MNIST data 
X = np.zeros((x_train.shape[0], 784))
for i in range(x_train.shape[0]):
    X[i] = x_train[i].flatten()
X = pd.DataFrame(X)
Y = pd.DataFrame(y_train)

# shuffle dataset and take random 20% for visualisation with tSNE 
X_sample = X.sample(frac=0.5, random_state=12).reset_index(drop=True)
Y_sample = Y.sample(frac=0.5, random_state=12).reset_index(drop=True)
X_sample.shape

(30000, 784)

### t-SNE 

In [7]:
# step 1: define affinities 
import openTSNE
affinites = openTSNE.affinity.PerplexityBasedNN(
    X_sample.to_numpy(),
    perplexity=50, 
    n_jobs=-1,
    random_state=12,
    verbose=True
    )

===> Finding 150 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 59.02 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 2.64 seconds


In [9]:
init = openTSNE.initialization.random(n_samples=len(X_sample) ,random_state=12, verbose=True)

In [10]:
embedding = openTSNE.TSNEEmbedding(
    init, 
    affinites, 
    verbose=True
)

## Custom Optimizer

In [11]:
def custom_optimize(
        self,
        n_iter,
        inplace=False,
        propagate_exception=False,
        **gradient_descent_params,
    ):
        """
        Returns
        -------
        TSNEEmbedding
            An optimized t-SNE embedding.

        Raises
        ------
        OptimizationInterrupt
            If a callback stops the optimization and the ``propagate_exception``
            flag is set, then an exception is raised.

        """
        # Typically we want to return a new embedding and keep the old one intact
        if inplace:
            embedding = self
        else:
            embedding = TSNEEmbedding(
                np.copy(self),
                self.affinities,
                random_state=self.random_state,
                optimizer=self.optimizer.copy(),
                **self.gradient_descent_params,
            )

        # If optimization parameters were passed to this funciton, prefer those
        # over the defaults specified in the TSNE object
        optim_params = dict(self.gradient_descent_params)
        optim_params.update(gradient_descent_params)
        optim_params["n_iter"] = n_iter
        _handle_nice_params(embedding, optim_params)

        try:
            # Run gradient descent with the embedding optimizer so gains are
            # properly updated and kept
            error, embedding = embedding.optimizer(
                embedding=embedding, P=self.affinities.P, **optim_params
            )

        except OptimizationInterrupt as ex:
            log.info("Optimization was interrupted with callback.")
            if propagate_exception:
                raise ex
            error, embedding = ex.error, ex.final_embedding

        embedding.kl_divergence = error

        return embedding

In [ ]:
custom_embedding = embedding.optimize(n_iter=250, learning_rate=200, exaggeration=12, momentum=0.5, callbacks= kld_tracker_EE, callbacks_every_iters=10, verbose=True)